In [1]:
import calendar
import pandas as pd
import itertools as it
from collections import Counter

# Предобработка

In [2]:
answers = {} # создадим словарь для ответов
data = pd.read_csv('movie_bd_v5.xls')

#в некоторых столбцах, пристуствуют значения разделенные |
#для того чтобы использовать explode и "разворачивать" значения стоит переделать их в списки
data['genres'] = data['genres'].str.split('|')
data['director'] = data['director'].str.split('|')
data['cast'] = data['cast'].str.split('|')
data['production_companies'] = data['production_companies'].str.split('|')

#Добавим два необходимых для задания столбца
data['profit'] = data['revenue'] - data['budget']  #прибыль с фильма
data['month'] = pd.to_datetime(data['release_date'], errors = 'coerce').dt.month
data['month'] = data['month'].apply(lambda x: calendar.month_abbr[x]) #месяц выхода

# 1. У какого фильма из списка самый большой бюджет?

In [3]:
answers[1] = data.loc[data['budget'].idxmax()].original_title

# 2. Какой из фильмов самый длительный (в минутах)?

In [4]:
answers[2] = data.loc[data['runtime'].idxmax()].original_title

# 3. Какой из фильмов самый короткий (в минутах)?





In [5]:
answers[3] = data.loc[data['runtime'].idxmin()].original_title

# 4. Какова средняя длительность фильмов?


In [6]:
answers[4] = data.runtime.mean()

# 5. Каково медианное значение длительности фильмов? 

In [7]:
answers[5] = data.runtime.median()

# 6. Какой самый прибыльный фильм?


In [8]:
answers[6] = data.loc[data['profit'].idxmax()].original_title

# 7. Какой фильм самый убыточный? 

In [9]:
answers[7] = data.loc[data['profit'].idxmin()].original_title

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [10]:
answers[8] = data[data['profit'] > 0].profit.count()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [11]:
answers[9] = data.loc[data[data['release_year'] == 2008].revenue.idxmax()].original_title

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [12]:
answers[10] = data.loc[data[(data['release_year'] >= 2012) 
                            & (data['release_year'] <= 2014)].profit.idxmin()].original_title

# 11. Какого жанра фильмов больше всего?

In [13]:
#для использования explode и различных других действий используем переменную temp_df
#чтобы не портить оригинальную переменную с датасетом
temp_df = data.explode('genres').reset_index()
answers[11] = temp_df.genres.value_counts().idxmax()

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [14]:
#тут используем датасет из прошлого вопроса
answers[12] = temp_df[temp_df['profit'] > 0].genres.value_counts().idxmax()

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [15]:
temp_df = data.explode('director').reset_index()
answers[13] = temp_df.groupby('director')['revenue'].sum().idxmax()

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [16]:
#минусом разделения значений на списоки, является то, что мы не можем использовать str.contains не потанцевав с бубном
#поэтому мы сначала меняем все списки в которых есть Action на Action, а потом уже фильруем датасет по строкам с Action
temp_df = temp_df[temp_df.genres.apply(lambda x: 'Action' if 'Action' in x else False).str.contains('Action', na=False)]
answers[14] = temp_df.groupby('director')['genres'].count().idxmax()

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [17]:
temp_df = data.explode('cast').reset_index()
temp_df = temp_df[temp_df['release_year'] == 2012]
answers[15] = temp_df.groupby('cast')['revenue'].sum().idxmax()

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [18]:
temp_df = data.explode('cast').reset_index()
answers[16] = temp_df[temp_df['budget'] > temp_df['budget'].mean()].cast.value_counts().idxmax()

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [19]:
temp_df = data.explode('genres').reset_index()
temp_df = temp_df[temp_df.cast.apply(lambda x: 'Nicolas Cage' if 'Nicolas Cage' in x else False)
    .str.contains('Nicolas Cage', na=False)]
answers[17] = temp_df.genres.value_counts().idxmax()

# 18. Самый убыточный фильм от Paramount Pictures

In [20]:
temp_df = data.explode('production_companies').reset_index()
temp_df = temp_df[temp_df.production_companies
    .apply(lambda x: 'Paramount Pictures' if 'Paramount Pictures' in x else False)
    .str.contains('Paramount Pictures', na=False)]
answers[18] = temp_df.loc[temp_df.profit.idxmin()].original_title

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [21]:
answers[19] = data.groupby('release_year')['revenue'].sum().idxmax()

# 20. Какой самый прибыльный год для студии Warner Bros?

In [22]:
temp_df = data.explode('production_companies').reset_index()
temp_df = temp_df[temp_df.production_companies
    .apply(lambda x: 'Warner Bros' if 'Warner Bros' in x else False)
    .str.contains('Warner Bros', na=False)]
answers[20] = temp_df.groupby('release_year')['profit'].sum().idxmax()    

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [23]:
answers[21] = data.groupby('month')['original_title'].count().idxmax()

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [24]:
answers[22] = data[(data['month']=='Jun')|(data['month']=='Jul')
                   | (data['month']=='Aug')].original_title.count()

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [25]:
temp_df = data.explode('director').reset_index()
temp_df = temp_df[(temp_df['month']=='Dec')
                  | (temp_df['month']=='Jan')|(temp_df['month']=='Feb')]
answers[23] = temp_df.groupby('director').imdb_id.count().idxmax()

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [26]:
temp_df = data.explode('production_companies').reset_index()
temp_df['name_len'] = temp_df.original_title.apply(lambda x: len(x))
answers[24] = temp_df.groupby('production_companies')['name_len'].mean().idxmax()

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [27]:
temp_df['overview_len'] = temp_df.overview.apply(lambda x: len(x))
answers[25] = temp_df.groupby('production_companies')['overview_len'].mean().idxmax()

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [28]:
#для того чтобы словарь с ответами выглядел нормально, датасет с 1% лучших засунул в отдельную переменную
df_26 = data.sort_values("vote_average", ascending = False).head(len(data.index)//100)
answers[26] = "Inside Out, The Dark Knight, 12 Years a Slave"

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [29]:
#набиваем список всеми комбинациями снимавшихся вместе актеров
temp_list = data.cast.apply(lambda x: list(it.combinations(x,2)))
#делаем из списка списков, один большой список
temp_list=[item for sublist in temp_list.values for item in sublist]
#ну и находим самую, часто попадающуюся пару
answers[27] = Counter(temp_list).most_common(2)

# Submission

In [30]:
answers

{1: 'Pirates of the Caribbean: On Stranger Tides',
 2: 'Gods and Generals',
 3: 'Winnie the Pooh',
 4: 109.6585494970884,
 5: 107.0,
 6: 'Avatar',
 7: 'The Lone Ranger',
 8: 1478,
 9: 'The Dark Knight',
 10: 'The Lone Ranger',
 11: 'Drama',
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker',
 19: 2015,
 20: 2014,
 21: 'Sep',
 22: 450,
 23: 'Peter Jackson',
 24: 'Four By Two Productions',
 25: 'Midnight Picture Show',
 26: 'Inside Out, The Dark Knight, 12 Years a Slave',
 27: [(('Daniel Radcliffe', 'Rupert Grint'), 8),
  (('Daniel Radcliffe', 'Emma Watson'), 8)]}

In [31]:
df_26 #датасет 26го вопроса

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",[Christopher Nolan],Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",7/16/2008,8.1,2008,816921825,Jul
118,tt0816692,165000000,621752480,Interstellar,"[Matthew McConaughey, Jessica Chastain, Anne H...",[Christopher Nolan],Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,"[Adventure, Drama, Science Fiction]","[Paramount Pictures, Legendary Pictures, Warne...",11/5/2014,8.0,2014,456752480,Nov
125,tt2084970,14000000,233555708,The Imitation Game,"[Benedict Cumberbatch, Keira Knightley, Matthe...",[Morten Tyldum],The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,"[History, Drama, Thriller, War]","[Black Bear Pictures, Bristol Automotive]",11/14/2014,8.0,2014,219555708,Nov
9,tt2096673,175000000,853708609,Inside Out,"[Amy Poehler, Phyllis Smith, Richard Kind, Bil...",[Pete Docter],Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,"[Comedy, Animation, Family]","[Walt Disney Pictures, Pixar Animation Studios...",6/9/2015,8.0,2015,678708609,Jun
34,tt3170832,6000000,35401758,Room,"[Brie Larson, Jacob Tremblay, Joan Allen, Sean...",[Lenny Abrahamson],Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,"[Drama, Thriller]","[Element Pictures, No Trace Camping, A24, Dupe...",10/16/2015,8.0,2015,29401758,Oct
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,"[Leonardo DiCaprio, Jonah Hill, Margot Robbie,...",[Martin Scorsese],EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,"[Crime, Drama, Comedy]","[Paramount Pictures, Appian Way, EMJAG Product...",12/25/2013,7.9,2013,292000694,Dec
128,tt2267998,61000000,369330363,Gone Girl,"[Ben Affleck, Rosamund Pike, Carrie Coon, Neil...",[David Fincher],You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,"[Mystery, Thriller, Drama]","[Twentieth Century Fox Film Corporation, Regen...",10/1/2014,7.9,2014,308330363,Oct
1191,tt2024544,20000000,187000000,12 Years a Slave,"[Chiwetel Ejiofor, Michael Fassbender, Lupita ...",[Steve McQueen],The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,"[Drama, History]","[Plan B Entertainment, Regency Enterprises, Ri...",10/18/2013,7.9,2013,167000000,Oct
119,tt2015381,170000000,773312399,Guardians of the Galaxy,"[Chris Pratt, Zoe Saldana, Dave Bautista, Vin ...",[James Gunn],All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,"[Action, Science Fiction, Adventure]","[Marvel Studios, Moving Picture Company (MPC),...",7/30/2014,7.9,2014,603312399,Jul
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,"[Elijah Wood, Ian McKellen, Viggo Mortensen, L...",[Peter Jackson],The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,"[Adventure, Fantasy, Action]","[WingNut Films, New Line Cinema]",12/1/2003,7.9,2003,1024888979,Dec
